# Spatially compounding drought and fire weather events for Western North America (WNA) and East Australia (EAU)

In [ ]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

On Pearcey, nodes have 20 cores, each with 6GB. More than 20 cores, need more than one worker. `processes` ensures the number of workers.

In [ ]:
cluster = SLURMCluster(processes=1,
                       walltime='01:00:00',
                       cores=3,
                       memory='18GB',
                       job_extra=['--qos="express"'])

In [ ]:
cluster.scale(jobs=1)

In [ ]:
client = Client(cluster)
client

In [ ]:
import xarray as xr
import numpy as np
import string
import copy

import matplotlib.pyplot as plt
import matplotlib as mpl

import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
cartopy.config['pre_existing_data_dir'] = '/datasets/work/oa-dcfp/work/squ027/data/cartopy-data'
cartopy.config['data_dir'] = '/datasets/work/oa-dcfp/work/squ027/data/cartopy-data'

In [ ]:
import functions as fn

# Plotting parameters

In [ ]:
fontsize = 7
coastlines_lw = 0.5
linewidth = 1.3
patheffect_lw_add = linewidth * 1.8
    
plt_params = {'lines.linewidth': linewidth,
              
              'hatch.linewidth': 0.5,
    
              'font.size': fontsize,
              
              'legend.fontsize' : fontsize-1,
              'legend.columnspacing': 0.7,
              'legend.labelspacing' : 0.03,
              'legend.handlelength' : 1.,
              'patch.linewidth': .5,
             
              'axes.linewidth': 0.5}

# default colours
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Process diagnostics

### JRA diagnostic data

u300, v300, z200, z500, vpot200

In [ ]:
jra_isobaric_ds = xr.open_zarr('/scratch1/projects/dcfp/data/csiro-dcfp-jra55/isobaric_month.zarr',
                               consolidated=True)
jra_isobaric_ds = jra_isobaric_ds.rename({'g0_lat_2': 'lat',
                                          'g0_lon_3': 'lon',
                                          'initial_time0_hours': 'time',
                                          'lv_ISBL1': 'level'})

In [ ]:
u300 = jra_isobaric_ds['UGRD_GDS0_ISBL'].sel(level=300).rename('u300').compute()
v300 = jra_isobaric_ds['VGRD_GDS0_ISBL'].sel(level=300).rename('v300').compute()

In [ ]:
z200 = jra_isobaric_ds['HGT_GDS0_ISBL'].sel(level=200).rename('z200').compute()

In [ ]:
z500 = jra_isobaric_ds['HGT_GDS0_ISBL'].sel(level=500).rename('z500').compute()

In [ ]:
vpot200 = jra_isobaric_ds['VPOT_GDS0_ISBL'].sel(level=200).rename('vpot200').compute()

#### MSLP

In [ ]:
jra_surface_ds = xr.open_zarr('/scratch1/projects/dcfp/data/csiro-dcfp-jra55/surface_month.zarr',
                              consolidated=True)

In [ ]:
mslp = jra_surface_ds['PRMSL_GDS0_MSL'].rename({'initial_time0_hours': 'time',
                                                'g0_lat_1': 'lat',
                                                'g0_lon_2': 'lon'})
mslp = mslp.rename('mslp').compute()

#### Combine

In [ ]:
jra_ds = xr.Dataset()
jra_ds['u300'] = u300
jra_ds['v300'] = v300
jra_ds['z200'] = z200
jra_ds['z500'] = z500
jra_ds['vpot200'] = vpot200
jra_ds['mslp'] = mslp

### Climatology and anomalies

In [ ]:
jra_clim = fn.climatology_monthly(jra_ds)
jra_anom = fn.anomalise_monthly(jra_ds)

### HadISST data

In [ ]:
hadisst = xr.open_zarr('/scratch1/projects/dcfp/data/csiro-dcfp-hadisst/ocean_month.zarr')
hadisst = hadisst['sst'].sel(time=slice('1958', '2020')).rename({'latitude': 'lat', 'longitude': 'lon'}).compute()
hadisst = hadisst.where(hadisst > -10, np.nan)

In [ ]:
sst_anom = fn.anomalise_monthly(hadisst, climatology_slice=slice('1958', '2020')).compute()

# Climate mode indices

Nino 3.4

In [ ]:
nino34 = sst_anom.sel(lat=slice(5, -5), lon=slice(-170, -120)).mean(['lat','lon'])
nino34_dt = fn.detrend_dim(nino34, 'time')

IOD

In [ ]:
dmi = fn.calc_dmi(sst_anom)
dmi_dt = fn.detrend_dim(dmi, 'time')

SAM

In [ ]:
sam = fn.calc_sam(jra_ds['mslp'], clim_period=[jra_ds.time[0], jra_ds.time[-1]])
sam_dt = fn.detrend_dim(sam, 'time')

GAR: Gulf of Alaska ridge index: https://www.nature.com/articles/ncomms9657#Sec7

In [ ]:
gar = jra_anom['z200'].sel(lat=slice(55, 45), lon=slice(-145+360, -135+360)).mean(['lat', 'lon'])
gar_dt = fn.detrend_dim(gar, 'time')

PNA

In [ ]:
pna = fn.calc_pna(jra_anom['z500'])
pna_dt = fn.detrend_dim(pna, 'time')

# Load FFDI, precipitation and burned area data

In [ ]:
# Regional extreme FFDI days per month
ar6_ffdi_m = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/jra_ar6_extreme_ffdi_dpm_global_195801_202012.zarr',
                         consolidated=True).ffdi_dpm
ar6_ffdi_m = ar6_ffdi_m.compute()

In [ ]:
# Regional monthly precip
ar6_pr = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/gpcc_ar6_precip_monthly_global_1950_2020.zarr',
                     consolidated=True).precip
ar6_pr = ar6_pr.compute()

In [ ]:
# Regional monthly burned area
ar6_ba = xr.open_zarr('/scratch1/ric368/projects/fire/data/derived/cci_ar6_burned_area_monthly_global_200101_202012.zarr',
                     consolidated=True).burned_area
ar6_ba = ar6_ba.compute()

# Which set of months should the modes be averaged over?

#### Correlation between 3 monthly fire season FFDI and modes - which month gives highest correlation?

In [ ]:
ffdi_wna = ar6_ffdi_m.sel(region='WNA').rolling(time=3).sum()
ffdi_eau = ar6_ffdi_m.sel(region='EAU').rolling(time=3).sum()

#### Choose month based on maximum 3-monthly FFDI

- JJA for WNA
- OND for EAU

In [ ]:
ffdi_wna.groupby('time.month').mean().plot()
ffdi_eau.groupby('time.month').mean().plot()

In [ ]:
modes = {'nino34': nino34,
         'dmi': dmi,
         'sam': sam.drop('level'),
         'gar': gar.drop('level'),
         'pna': pna.drop('level')}

In [ ]:
# Calculate correlation between FFDI and modes

roll = 3

ffdi_mode_cor = np.full((2, 12, len(modes.keys())), np.nan)

for i, ffdi_da, ffdi_season in zip([0,1], [ffdi_wna, ffdi_eau], [8, 12]):
    f_ = ffdi_da.rolling(time=3).sum()
    for j, mode in enumerate(list(modes.keys())):
        m_ = modes[mode].rolling(time=roll).mean()
        
        for month in range(1,13):
            f = f_.isel(time=f_.groupby('time.month').groups[ffdi_season])                
            m = m_.isel(time=m_.groupby('time.month').groups[month])
            m = m.assign_coords({'time': f.time})
            ffdi_mode_cor[i,month-1,j] = xr.corr(f, m, 'time').values

In [ ]:
# Put into DataArray
ffdi_mode_cor = xr.DataArray(ffdi_mode_cor,
                             coords={'region': ['WNA', 'EAU'],
                                     'month': range(1,13),
                                     'mode': list(modes.keys())},
                             dims=['region',
                                   'month',
                                   'mode'])

In [ ]:
mode_titles = [r'Ni$\mathrm{\~n}$o3.4',
               'DMI',
               'SAM',
               'GAR',
               'PNA']

with plt.rc_context(plt_params):
    fig, ax = plt.subplots(2, 3, figsize=(6.9, 3))

    for i, mode in enumerate(ffdi_mode_cor.mode):
        for j, r in enumerate(ffdi_mode_cor.region):
            d = ffdi_mode_cor.sel(region=r, mode=mode)

            p = d.plot(ax=ax.flatten()[i], label=r.values)
            ax.flatten()[i].axvline(np.where(abs(d) == abs(d).max())[0]+1, c=colors[j], alpha=0.2, lw=10)
            ax.flatten()[i].set_title('')
            ax.flatten()[i].text(0.03, 0.87, mode_titles[i], transform=ax.flatten()[i].transAxes)
            ax.flatten()[i].axhline(0, ls='--', c='k')
            ax.flatten()[i].set_xticks(range(1,13))
            ax.flatten()[i].set_yticks(np.arange(-0.4, 0.41, 0.2))
            ax.flatten()[i].set_xlabel('Month')
            ax.flatten()[i].set_ylabel('Pearson correlation [-]')
        if i == 0:
            ax.flatten()[i].legend(loc=4)

    ax[-1,-1].axis('off')
    plt.tight_layout()
    
#     plt.savefig('./figures/mode_ffdi_correlation.pdf', bbox_inches='tight', format='pdf', dpi=400)

# Average modes over months of greatest correlation with FFDI, except PNA

In [ ]:
OND_nino34 = nino34.rolling(time=3).mean()
OND_nino34 = OND_nino34.isel(time=OND_nino34.groupby('time.month').groups[12])

In [ ]:
SON_dmi = dmi.rolling(time=3).mean()
SON_dmi = SON_dmi.isel(time=SON_dmi.groupby('time.month').groups[11])

In [ ]:
ASO_sam = sam.rolling(time=3).mean()
ASO_sam = ASO_sam.isel(time=ASO_sam.groupby('time.month').groups[10])

In [ ]:
ASO_gar = gar.rolling(time=3).mean()
ASO_gar = ASO_gar.isel(time=ASO_gar.groupby('time.month').groups[10])

In [ ]:
DJF_pna = pna.rolling(time=3).mean()
DJF_pna = DJF_pna.isel(time=DJF_pna.groupby('time.month').groups[2])

In [ ]:
modes = {r'Ni$\mathrm{\~n}$o3.4 OND': OND_nino34 / OND_nino34.std('time'),
        'DMI SON': SON_dmi / SON_dmi.std('time'),
        'SAM ASO': ASO_sam / ASO_sam.std('time'),
        'PNA DJF': DJF_pna / DJF_pna.std('time'),
        'GAR ASO': ASO_gar / ASO_gar.std('time')}

# Figure 6: Time series of FFDI, precip, burned area and modes

In [ ]:
def intersperse(lst, item, truncate=False):
    if truncate:
        result = [item] * (len(lst) * 2 - 1)
    else:
        result = [item] * (len(lst) * 2)
    result[0::2] = lst
    return result

In [ ]:
detrend = False
modes_array = np.full((len(ASO_sam.time), len(modes.keys())), np.nan)
for i, v in enumerate(modes.values()):
    if detrend:
        modes_array[:,i] = fn.detrend_dim(v, 'time').values
    else:
        modes_array[:,i] = v.values

In [ ]:
np.nanmin(modes_array)

In [ ]:
np.nanmax(modes_array)

In [ ]:
def extend_event_months(event_da, aggregation_length):
    """
        Returns a binary dataArray indicating time points prior to events
        that contributed to that event. E.g. for 12-month rainfall events
        that constitude a drought, the preceding 11 months are labelled 1.
    """
    d = xr.where(event_da.notnull(), 1, 0)
    event_start = np.where(d.diff('time').values == 1)[0]
    
    prior_indices = []
    for i in range(1,aggregation_length):
        prior_indices.append(event_start - i)
    indices = np.sort(np.unique(np.concatenate(prior_indices)))
    
    all_dates = np.full(len(d), np.nan)
    all_dates[indices] = 1
    
    return xr.DataArray(all_dates, coords=d.coords, dims=d.dims)

In [ ]:
def lego_modes(ffdi_da, ba_da, pr_da, m_array,
               r1='WNA', r2='EAU', fy=1958, ly=2020,
               fw_roll=3, pr_roll=12, min_months=3, q=0.9,
               save_plot=True, filename='spatially_compounding_modes.pdf'):
    
    years = range(fy, ly+1)
    
    # ========= Prep data
    # FFDI
    # Shift data backwards 3 months so that Jan-Dec now actually shows Apr-Mar
    ffdi_da1 = ffdi_da.sel(region=r1).rolling(time=fw_roll).sum().shift(time=-3)
    ffdi_da2 = ffdi_da.sel(region=r2).rolling(time=fw_roll).sum().shift(time=-3)

    # Burned area
    ba_da1 = ba_da.sel(region=r1).rolling(time=fw_roll).sum().shift(time=-3)
    ba_da2 = ba_da.sel(region=r2).rolling(time=fw_roll).sum().shift(time=-3)

    # Precip
    pr_da1 = pr_da.sel(region=r1).rolling(time=pr_roll).sum().shift(time=-3)
    pr_da2 = pr_da.sel(region=r2).rolling(time=pr_roll).sum().shift(time=-3)

    # Thresholds for events
    ffdi_t_da1 = ffdi_da1.mean('time') + ffdi_da1.std('time') #ffdi_da1.quantile(q)
    ffdi_t_da2 = ffdi_da2.mean('time') + ffdi_da2.std('time') #ffdi_da2.quantile(q)

    ba_t_da1 = ba_da1.mean('time') + ba_da1.std('time')  #ba_da1.quantile(q)
    ba_t_da2 = ba_da2.mean('time') + ba_da2.std('time')  #ba_da2.quantile(q)

    pr_t_da1 = pr_da1.mean('time') - pr_da1.std('time')   #pr_da1.quantile(1-q)
    pr_t_da2 = pr_da2.mean('time') - pr_da2.std('time')   #pr_da2.quantile(1-q)
    
    # Find years that start or end on a drought
    start_drought1 = (pr_da1 < pr_t_da1).isel(time=pr_da1.groupby('time.month').groups[1])
    start_drought2 = (pr_da2 < pr_t_da2).isel(time=pr_da2.groupby('time.month').groups[1])
    end_drought1 = (pr_da1 < pr_t_da1).isel(time=pr_da1.groupby('time.month').groups[12])
    end_drought2 = (pr_da2 < pr_t_da2).isel(time=pr_da2.groupby('time.month').groups[12])

    with plt.rc_context(plt_params):

        tick_length = 2
        tick_width = .5
        xticklabels = []
        
        da1_color = colors[0] #cm.get_cmap('magma')(0.3)
        da2_color = colors[1]
        both_color = colors[6]

        fig = plt.figure(figsize=(7, 2))
        gs = fig.add_gridspec(ncols=len(years), nrows=2, figure=fig,height_ratios=[1,1])

        end_year_drought1 = 0 # 1 == end on drought, 0 otherwise
        end_year_drought2 = 0
        
        for i, y in enumerate(years):
            ax = fig.add_subplot(gs[0,i])

            # ========== FFDI
            y1 = range(len(ffdi_da1.sel(time=str(y)).time))
            x1 = xr.where(ffdi_da1.sel(time=str(y)) > ffdi_t_da1, 1, 0)
            p_ffdi1 = ax.barh(y1, x1, color=da1_color, alpha=0.5, zorder=1, label=r1+' FWE')

            y2 = range(len(ffdi_da2.sel(time=str(y)).time))
            x2 = xr.where(ffdi_da2.sel(time=str(y)) > ffdi_t_da2, 1, 0)
            p_ffdi2 = ax.barh(y2, x2, color=da2_color, alpha=0.5, zorder=1, label=r2+' FWE')
            
            x3 = xr.where((x1 + x2) == 2, 1, 0)
            ax.barh(y2, x3, color='w', zorder=1)
            p_both = ax.barh(y2, x3, color=both_color, alpha=.55, zorder=2, label='Both FWE')
            
            # If number of FFDI events exceeds min_months for either/both regions, plot marker
            x1_tot = x1.sum().values
            x2_tot = x2.sum().values
            
            ec = 'none'
            ms = 15
            alpha = 0.8
            
            if (x1_tot >= min_months) & (x2_tot < min_months):
                ax.scatter(0.5, y1[-1]+1.3, marker='v', s=ms, color=da1_color, alpha=alpha, edgecolor=ec, lw=.05, clip_on=False)
                
            if (x2_tot >= min_months) & (x1_tot < min_months):
                ax.scatter(0.5, y1[-1]+1.3, marker='^', s=ms, color=da2_color, alpha=alpha, edgecolor=ec, lw=.05, clip_on=False)
                
            if (x1_tot >= min_months) & (x2_tot >= min_months):
                ax.scatter(0.5, y1[-1]+1.3, marker='s', s=ms, color=both_color, alpha=alpha, edgecolor=ec, lw=.05, clip_on=False, label='XXX')

            # ========== Burned area
            if y > 2000:
                x1 = xr.where(ba_da1.sel(time=str(y)) > ba_t_da1, 0.5, np.nan)
                p_ba1 = ax.scatter(x1, y1, marker='o', lw=.5, s=10, facecolor='None', edgecolor=da1_color, alpha=1, zorder=5, label=r1+' burned area')

                x2 = xr.where(ba_da2.sel(time=str(y)) > ba_t_da2, 0.5, np.nan)
                ax.scatter(x2, y2, marker='o', s=6, facecolor='white', edgecolor='None', alpha=1, zorder=3)
                p_ba2 = ax.scatter(x2, y2, marker='o', s=3, facecolor=da2_color, edgecolor='None', alpha=1, zorder=4, label=r2+' burned area')

            # ========== Precip
            x1 = xr.where(pr_da1.sel(time=str(y)) < pr_t_da1, 0.8, np.nan)
            x2 = xr.where(pr_da2.sel(time=str(y)) < pr_t_da2, 0.2, np.nan)
                
            # If drought extends into this year, extend the line backwards
            if end_year_drought1 == 1:
                y1 = np.append(-1, y1)
                x1 = np.append(0.8, x1)
            if end_year_drought2 == 1:
                y2 = np.append(-1, y2)
                x2 = np.append(0.2, x2)
                
            if y < 2020:
                # Set whether this year ends on a drought
                end_year_drought1 = end_drought1.sel(time=str(y)).values # 1 == end on drought, 0 otherwise
                end_year_drought2 = end_drought2.sel(time=str(y)).values

                # If drought extends into next year, extend the line forwards
                next_year_drought1 = start_drought1.sel(time=str(y+1)).values
                next_year_drought2 = start_drought2.sel(time=str(y+1)).values
                if next_year_drought1 == 1:
                    y1 = np.append(y1, np.max(y1)+1)
                    x1 = np.append(x1, 0.8)
                if next_year_drought2 == 1:
                    y2 = np.append(y2, np.max(y2)+1)
                    x2 = np.append(x2, 0.2)

            p_pr1 = ax.plot(x1, y1, color=da1_color, marker='x', ms=1.5, markeredgewidth=.5, lw=.5, alpha=1, zorder=2, label=r1+' drought')
            p_pr2 = ax.plot(x2, y2, color=da2_color, marker='x', ms=1.5, markeredgewidth=.5, lw=.5, alpha=1, zorder=2, label=r2+' drought')
            
            # Fill JFM in grey as data ends in 2020
            if y == 2020:
                ax.axhspan(y1[-3]-.5, y1[-1]+.5, color='gray')

            ax.set_xlim(0, 1)
            ax.set_xticks([0.5])
            
            ax.xaxis.tick_top()
            if y % 5 == 0:
                ax.tick_params(axis='x', length=tick_length * 2, width=tick_width)
                ax.set_xticklabels([str(y)+'/'+str(y+1)[-2:]], fontsize=plt_params['font.size']-1)
                xticklabels.append(y)
            else:
                ax.tick_params(axis='x', length=tick_length, width=tick_width)
                ax.set_xticklabels('')
                xticklabels.append('')

            ax.set_ylim(-0.4, 11.4)
            ax.set_yticks([])
            ax.set_yticklabels('')  
            if i == 0:
                yticks = range(12)
                ax.set_yticks(yticks)
                ax.tick_params(axis='y', length=tick_length, width=tick_width)
                ax.set_yticklabels(['Apr', '', 'Jun', '', 'Aug', '', 'Oct', '', 'Dec', '', 'Feb', ''], fontsize=plt_params['font.size']-1)
                ax.set_ylabel('Month')
                ax.plot((-1.5,0.), (12.3, 12.3), color='k', lw=tick_width, clip_on=False)
                ax.text(-1.6, 12.2, '$\geq3$ FWE', fontsize=plt_params['font.size']-1, clip_on=False, ha='right', va='center')
                
            ax.invert_yaxis()

            if y == 2020:
                yticks = range(12)
                ax.legend(ncol=4, loc='lower left', bbox_to_anchor=(-62.5, -1.6, 42, 1), mode='expand')
                ax.set_yticks(yticks)
                ax.tick_params(axis='y', length=tick_length, width=tick_width)
                ax.set_yticklabels(['Apr', '', 'Jun', '', 'Aug', '', 'Oct', '', 'Dec', '', 'Feb', ''], fontsize=plt_params['font.size']-1)
                ax.yaxis.tick_right()

        ### ========================= Modes plot
        current_cmap = copy.copy(mpl.cm.get_cmap('RdBu_r'))
        current_cmap.set_bad('gray')
        ax2 = fig.add_subplot(gs[1,:])
        p = ax2.imshow(m_array.T, cmap=current_cmap, vmin=-2.7, vmax=2.7)

        ax2.set_xticks(range(len(years)))
        ax2.tick_params(axis='both', length=tick_length, width=tick_width)
        ax2.set_xticklabels(xticklabels, fontsize=plt_params['font.size']-1)
        ax2.set_xlabel('Year')

        ax2.set_yticks(range(len(modes.keys())))
        ax2.set_yticklabels(modes.keys(), rotation=0, fontsize=plt_params['font.size']-1)
        ax2.yaxis.set_ticks_position('both')
        ax2.tick_params(labelright=True)
        
        cb_ax = fig.add_axes([0.65, 0.015, 0.25, 0.03])
        cb = fig.colorbar(p, cax=cb_ax, orientation='horizontal', ticks=[-2.7, -1.8, -0.9, 0, 0.9, 1.8, 2.7], extend='neither')
        cb.set_label('Anomaly [-]', fontsize=plt_params['font.size']-1)
        cb.ax.tick_params(labelsize=plt_params['font.size']-1) 

        plt.subplots_adjust(wspace=0.0, hspace=0.)
        
        if save_plot:
            plt.savefig('./figures/'+filename, bbox_inches='tight', format='pdf', dpi=400)

In [ ]:
%%time
lego_modes(ar6_ffdi_m, ar6_ba, ar6_pr, modes_array, save_plot=True)

# Close cluster

In [ ]:
client.close()
cluster.close()